In [113]:
import sys
import os

# Get the current working directory
current_dir = os.getcwd()

# Add the parent directory to sys.path
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.insert(0, parent_dir)

In [114]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from urllib.request import urlretrieve
from scipy.stats import pearsonr

from config.config import DATA_PATH

In [115]:
pd.set_option('display.max_columns', None)

df = pd.read_csv(DATA_PATH + '/preprocessed/airline_performance.csv', parse_dates=['scheduled_departure_datetime', 'scheduled_arrival_datetime', 'scheduled_departure_datetime_utc', 'scheduled_arrival_datetime_utc'])

In [116]:
df.head()

,year,quarter,month,day_of_month,day_of_week,marketing_airline_id,flight_number_marketing_airline,operating_airline_id,tail_number,origin_airport_id,origin_city_market_id,origin,origin_state,dest_airport_id,dest_city_market_id,dest,dest_state,dep_delay,dep_del15,departure_delay_groups,taxi_out,taxi_in,arr_delay,arr_del15,arrival_delay_groups,cancelled,diverted,scheduled_elapsed_time,actual_elapsed_time,air_time,distance,distance_group,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,div_airport_landings,code_share_flight,origin_city,destination_city,scheduled_departure_datetime,scheduled_arrival_datetime,actual_departure_datetime,actual_arrival_datetime,overnight_depature,overnight_arrival,airline_mkt,airline_ops,origin_timezone,destination_timezone,scheduled_departure_datetime_utc,actual_departure_datetime_utc,scheduled_arrival_datetime_utc,actual_arrival_datetime_utc,is_holiday,route_id,hour_of_day,departure_window,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_median_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,arr_delay_median_30D,cancelled_sum_30D,div_airport_landings_sum_30D,dep_delay_max_90D,cancelled_sum_90D,div_airport_landings_sum_90D,departure_status,arrival_status,region_origin,division_origin,region_dest,division_dest,temperature_2m_origin,precipitation_origin,snow_depth_origin,weather_code_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snow_depth_dest,weather_code_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest
0,2023,3,8,6,7,20416,2252,20416,N978NK,12889,32211,LAS,NV,11057,31057,CLT,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,263.0,NaN,NaN,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-06 23:59:00,2023-08-07 07:22:00,NaN,NaN,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-07 07:00:00+00:00,NaN,2023-08-07 11:00:00+00:00,NaN,0,3228,23,late night,5.125000,2.0,31.0,-6.0,3.0,0.0,8.0,15.518519,5.0,82.0,0.5,4.0,0.0,163.0,5.0,0.0,cancelled,cancelled,West,Mountain,South,South Atlantic,87.6,0.0,0.0,0.0,293635.188,5.1,180.0,8.5,72.6,0.0,0.0,3.0,40026.246,4.2,196.0,8.5
1,2023,3,8,7,1,20416,2252,20416,N974NK,12889,32211,LAS,NV,11057,31057,CLT,NC,76.0,1.0,5.0,19.0,9.0,69.0,1.0,4.0,0,0,263.0,256.0,228.0,1916.0,8,3.0,0.0,1.0,0.0,65.0,0.0,0,Las Vegas,Charlotte,2023-08-07 23:59:00,2023-08-08 07:22:00,2023-08-08 01:15:00,2023-08-08 08:31:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-08 07:00:00+00:00,2023-08-08 08:15:00+00:00,2023-08-08 11:00:00+00:00,2023-08-08 12:31:00+00:00,0,3228,23,late night,14.000000,2.0,76.0,-6.0,3.0,0.0,8.0,17.740741,5.0,82.0,5.0,4.0,0.0,163.0,5.0,0.0,delayed,delayed,West,Mountain,South,South Atlantic,89.7,0.0,0.0,0.0,293635.188,13.5,220.0,25.3,70.0,0.0,0.0,3.0,42322.836,4.9,223.0,18.1
2,2023,3,8,9,3,20416,2252,20416,N519NK,12889,32211,LAS,NV,11057,31057,CLT,NC,-11.0,0.0,-1.0,14.0,10.0,-13.0,0.0,-1.0,0,0,258.0,256.0,232.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-09 23:10:00,2023-08-10 06:28:00,2023-08-09 22:59:00,2023-08-10 06:15:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-10 06:00:00+00:00,2023-08-10 05:59:00+00:00,2023-08-10 10:00:00+00:00,2023-08-10 10:15:00+00:00,0,3228,23,late night,12.375000,1.5,76.0,-6.0,3.0,0.0,8.0,15.185185,5.0,82.0,0.5,4.0,0.0,163.0,5.0,0.0,early,early,West,Mountain,South,South Atlantic,91.9,0.0,0.0,3.0,234251.969,11.7,239.0,22.6,72.9,0.0,0.0,3.0,52821.523,3.4,212.0,9.4
3,2023,3,8,10,4,20416,2252,20416,N532NK,12889,32211,LAS,NV,11057,31057,CLT,NC,-8.0,0.0,-1.0,12.0,7.0,-30.0,0.0,-2.0,0,0,258.0,236.0,217.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-10 23:10:00,2023-08-11 06:28:00,2023-08-10 23:02:00,2023-08-11 05:58:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_An

In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1493764 entries, 0 to 1493763
Data columns (total 97 columns):
 #   Column                            Non-Null Count    Dtype              
---  ------                            --------------    -----              
 0   year                              1493764 non-null  int64              
 1   quarter                           1493764 non-null  int64              
 2   month                             1493764 non-null  int64              
 3   day_of_month                      1493764 non-null  int64              
 4   day_of_week                       1493764 non-null  int64              
 5   marketing_airline_id              1493764 non-null  int64              
 6   flight_number_marketing_airline   1493764 non-null  int64              
 7   operating_airline_id              1493764 non-null  int64              
 8   tail_number                       1488178 non-null  object             
 9   origin_airport_id                 1

In [118]:
df.shape

(1493764, 97)

In [119]:
df.columns.to_list()

['year',
 'quarter',
 'month',
 'day_of_month',
 'day_of_week',
 'marketing_airline_id',
 'flight_number_marketing_airline',
 'operating_airline_id',
 'tail_number',
 'origin_airport_id',
 'origin_city_market_id',
 'origin',
 'origin_state',
 'dest_airport_id',
 'dest_city_market_id',
 'dest',
 'dest_state',
 'dep_delay',
 'dep_del15',
 'departure_delay_groups',
 'taxi_out',
 'taxi_in',
 'arr_delay',
 'arr_del15',
 'arrival_delay_groups',
 'cancelled',
 'diverted',
 'scheduled_elapsed_time',
 'actual_elapsed_time',
 'air_time',
 'distance',
 'distance_group',
 'carrier_delay',
 'weather_delay',
 'nas_delay',
 'security_delay',
 'late_aircraft_delay',
 'div_airport_landings',
 'code_share_flight',
 'origin_city',
 'destination_city',
 'scheduled_departure_datetime',
 'scheduled_arrival_datetime',
 'actual_departure_datetime',
 'actual_arrival_datetime',
 'overnight_depature',
 'overnight_arrival',
 'airline_mkt',
 'airline_ops',
 'origin_timezone',
 'destination_timezone',
 'scheduled_d

## Columns to Drop


In [120]:
cols_to_keep = [
    'quarter',
    'month',
    'day_of_month',
    'day_of_week',
    'hour_of_day',
    'origin',
    'dest',
    'cancelled',
    'scheduled_elapsed_time',
    'distance',
    'code_share_flight',
    'scheduled_departure_datetime',
    'scheduled_arrival_datetime',
    'airline_mkt',
    'airline_ops',
    'is_holiday',
    'route_id',
    'departure_window',
    'dep_delay_mean_10D',
    'dep_delay_median_10D',
    'dep_delay_max_10D',
    'arr_delay_median_10D',
    'cancelled_sum_10D',
    'div_airport_landings_sum_10D',
    'n_flights_10D',
    'dep_delay_mean_30D',
    'dep_delay_median_30D',
    'dep_delay_max_30D',
    'arr_delay_median_30D',
    'cancelled_sum_30D',
    'div_airport_landings_sum_30D',
    'cancelled_sum_90D',
    'div_airport_landings_sum_90D',
    'region_origin',
    'division_origin',
    'region_dest',
    'division_dest',
    'temperature_2m_origin',
    'precipitation_origin',
    'snow_depth_origin',
    'weather_code_origin',
    'visibility_origin',
    'wind_speed_10m_origin',
    'wind_direction_10m_origin',
    'wind_gusts_10m_origin',
    'temperature_2m_dest',
    'precipitation_dest',
    'snow_depth_dest',
    'weather_code_dest',
    'visibility_dest',
    'wind_speed_10m_dest',
    'wind_direction_10m_dest',
    'wind_gusts_10m_dest']

flights_top20 = df[cols_to_keep]

## Missing Values

In [121]:
missing_values = flights_top20.isna().sum().reset_index(name='count')
missing_values[missing_values['count'] > 0].sort_values('count', ascending=False).head(60)

,index,count
52,wind_gusts_10m_dest,1317
51,wind_direction_10m_dest,1317
50,wind_speed_10m_dest,1317
49,visibility_dest,1317
48,weather_code_dest,1317
47,snow_depth_dest,1317
46,precipitation_dest,1317
45,temperature_2m_dest,1317
41,visibility_origin,708
44,wind_gusts_10m_origin,708


In [122]:
flights_top20[['cancelled']].sum() / len(flights_top20)

cancelled    0.014416
dtype: float64

In [123]:
flights_top20[flights_top20['wind_gusts_10m_dest'].isna()]['cancelled'].sum()

1

In [124]:
flights_missing_dropped = flights_top20.dropna(subset=['wind_gusts_10m_dest'])
missing_values = flights_missing_dropped.isna().sum().reset_index(name='count')
missing_values[missing_values['count'] > 0].sort_values('count', ascending=False).head(60)

,index,count
21,arr_delay_median_10D,195
18,dep_delay_mean_10D,188
19,dep_delay_median_10D,188
20,dep_delay_max_10D,188
28,arr_delay_median_30D,173
25,dep_delay_mean_30D,166
26,dep_delay_median_30D,166
27,dep_delay_max_30D,166
8,scheduled_elapsed_time,1


In [125]:
# calculate the average aggregated performance by airline_mkt and route_id
route_airline_agg = flights_missing_dropped.groupby(['airline_mkt', 'route_id']).agg({
    'dep_delay_mean_10D':'mean',
    'dep_delay_median_10D':'mean',
    'dep_delay_max_10D':'mean',
    'arr_delay_median_10D':'mean',
    'cancelled_sum_10D':'mean',
    'div_airport_landings_sum_10D':'mean',
    'n_flights_10D':'mean',
    'dep_delay_mean_30D':'mean',
    'dep_delay_median_30D':'mean',
    'dep_delay_max_30D':'mean',
    'arr_delay_median_30D':'mean',
    'cancelled_sum_30D':'mean',
    'div_airport_landings_sum_30D':'mean',
    'cancelled_sum_90D':'mean',
    'div_airport_landings_sum_90D':'mean',
    
}).reset_index()

agg_cols = route_airline_agg.drop(columns=['airline_mkt', 'route_id']).columns.to_list()
agg_cols
# merge the aggregated data back into the missing stats to fill missing values
missing_stats_filled = pd.merge(
    flights_missing_dropped,
    route_airline_agg,
    on=['airline_mkt', 'route_id'],
    suffixes=('', '_agg'),
    how='left'
)

# Fill missing values in the original columns with the aggregated values
for col in agg_cols:
    missing_stats_filled[col] = missing_stats_filled[col].fillna(missing_stats_filled[col + '_agg'])

# Drop the extra columns used for merging
missing_stats_filled = missing_stats_filled.drop(columns=[col + '_agg' for col in agg_cols])
missing_stats_filled.head()

,quarter,month,day_of_month,day_of_week,hour_of_day,origin,dest,cancelled,scheduled_elapsed_time,distance,code_share_flight,scheduled_departure_datetime,scheduled_arrival_datetime,airline_mkt,airline_ops,is_holiday,route_id,departure_window,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_median_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,arr_delay_median_30D,cancelled_sum_30D,div_airport_landings_sum_30D,cancelled_sum_90D,div_airport_landings_sum_90D,region_origin,division_origin,region_dest,division_dest,temperature_2m_origin,precipitation_origin,snow_depth_origin,weather_code_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snow_depth_dest,weather_code_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest
0,3,8,6,7,23,LAS,CLT,1,263.0,1916.0,0,2023-08-06 23:59:00,2023-08-07 07:22:00,Spirit Air Lines,Spirit Air Lines,0,3228,late night,5.125000,2.0,31.0,-6.0,3.0,0.0,8.0,15.518519,5.0,82.0,0.5,4.0,0.0,5.0,0.0,West,Mountain,South,South Atlantic,87.6,0.0,0.0,0.0,293635.188,5.1,180.0,8.5,72.6,0.0,0.0,3.0,40026.246,4.2,196.0,8.5
1,3,8,7,1,23,LAS,CLT,0,263.0,1916.0,0,2023-08-07 23:59:00,2023-08-08 07:22:00,Spirit Air Lines,Spirit Air Lines,0,3228,late night,14.000000,2.0,76.0,-6.0,3.0,0.0,8.0,17.740741,5.0,82.0,5.0,4.0,0.0,5.0,0.0,West,Mountain,South,South Atlantic,89.7,0.0,0.0,0.0,293635.188,13.5,220.0,25.3,70.0,0.0,0.0,3.0,42322.836,4.9,223.0,18.1
2,3,8,9,3,23,LAS,CLT,0,258.0,1916.0,0,2023-08-09 23:10:00,2023-08-10 06:28:00,Spirit Air Lines,Spirit Air Lines,0,3228,late night,12.375000,1.5,76.0,-6.0,3.0,0.0,8.0,15.185185,5.0,82.0,0.5,4.0,0.0,5.0,0.0,West,Mountain,South,South Atlantic,91.9,0.0,0.0,3.0,234251.969,11.7,239.0,22.6,72.9,0.0,0.0,3.0,52821.523,3.4,212.0,9.4
3,3,8,10,4,23,LAS,CLT,0,258.0,1916.0,0,2023-08-10 23:10:00,2023-08-11 06:28:00,Spirit Air Lines,Spirit Air Lines,0,3228,late night,10.111111,1.0,76.0,-8.5,2.0,0.0,9.0,11.851852,3.0,76.0,-0.5,4.0,0.0,5.0,0.0,West,Mountain,South,South Atlantic,87.6,0.0,0.0,0.0,173884.516,5.9,18.0,9.4,68.7,0.0,0.0,0.0,44947.508,1.1,90.0,1.6
4,3,8,12,6,23,LAS,CLT,0,258.0,1916.0,0,2023-08-12 23:10:00,2023-08-13 06:28:00,Spirit Air Lines,Spirit Air Lines,0,3228,late night,12.250000,3.0,76.0,-5.0,1.0,0.0,8.0,12.961538,4.5,76.0,0.0,4.0,0.0,5.0,0.0,West,Mountain,South,South Atlantic,92.2,0.0,0.0,1.0,265091.875,10.2,50.0,13.0,69.0,0.0,0.0,0.0,40354.332,3.1,176.0,6.5


In [126]:
missing_values = missing_stats_filled.isna().sum().reset_index(name='count')
missing_values[missing_values['count'] > 0].sort_values('count', ascending=False).head(60)

,index,count
8,scheduled_elapsed_time,1


In [127]:
missing_time = missing_stats_filled[missing_stats_filled['scheduled_elapsed_time'].isna()]
missing_time['scheduled_elapsed_time'] = (pd.to_datetime(missing_time['scheduled_arrival_datetime']) - pd.to_datetime(missing_time['scheduled_departure_datetime'])).dt.total_seconds() / 60
missing_time

/var/folders/b2/nkpmrw0s42q_zm32rwshljyr0000gn/T/ipykernel_55758/1857774745.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_time['scheduled_elapsed_time'] = (pd.to_datetime(missing_time['scheduled_arrival_datetime']) - pd.to_datetime(missing_time['scheduled_departure_datetime'])).dt.total_seconds() / 60


,quarter,month,day_of_month,day_of_week,hour_of_day,origin,dest,cancelled,scheduled_elapsed_time,distance,code_share_flight,scheduled_departure_datetime,scheduled_arrival_datetime,airline_mkt,airline_ops,is_holiday,route_id,departure_window,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_median_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,arr_delay_median_30D,cancelled_sum_30D,div_airport_landings_sum_30D,cancelled_sum_90D,div_airport_landings_sum_90D,region_origin,division_origin,region_dest,division_dest,temperature_2m_origin,precipitation_origin,snow_depth_origin,weather_code_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snow_depth_dest,weather_code_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest
1420559,1,3,19,7,10,SFO,LAX,1,89.0,337.0,1,2023-03-19 10:20:00,2023-03-19 11:49:00,Alaska Airlines Inc.,SkyWest Airlines Inc.,0,5963,morning,4.296296,-2.0,69.0,-8.0,3.0,0.0,27.0,7.341176,-3.0,342.0,-12.0,4.0,0.0,6.0,0.0,West,Pacific,West,Pacific,50.8,0.0,0.0,3.0,43635.172,2.7,90.0,11.6,59.6,0.0,0.0,3.0,67585.305,4.8,221.0,11.4


In [128]:
missing_stats_filled['scheduled_elapsed_time'] = (pd.to_datetime(missing_stats_filled['scheduled_arrival_datetime']) - pd.to_datetime(missing_stats_filled['scheduled_departure_datetime'])).dt.total_seconds() / 60

In [129]:
missing_values = missing_stats_filled.isna().sum().reset_index(name='count')
missing_values[missing_values['count'] > 0].sort_values('count', ascending=False)

,index,count


In [130]:
missing_stats_filled.head()

,quarter,month,day_of_month,day_of_week,hour_of_day,origin,dest,cancelled,scheduled_elapsed_time,distance,code_share_flight,scheduled_departure_datetime,scheduled_arrival_datetime,airline_mkt,airline_ops,is_holiday,route_id,departure_window,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_median_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,arr_delay_median_30D,cancelled_sum_30D,div_airport_landings_sum_30D,cancelled_sum_90D,div_airport_landings_sum_90D,region_origin,division_origin,region_dest,division_dest,temperature_2m_origin,precipitation_origin,snow_depth_origin,weather_code_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snow_depth_dest,weather_code_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest
0,3,8,6,7,23,LAS,CLT,1,443.0,1916.0,0,2023-08-06 23:59:00,2023-08-07 07:22:00,Spirit Air Lines,Spirit Air Lines,0,3228,late night,5.125000,2.0,31.0,-6.0,3.0,0.0,8.0,15.518519,5.0,82.0,0.5,4.0,0.0,5.0,0.0,West,Mountain,South,South Atlantic,87.6,0.0,0.0,0.0,293635.188,5.1,180.0,8.5,72.6,0.0,0.0,3.0,40026.246,4.2,196.0,8.5
1,3,8,7,1,23,LAS,CLT,0,443.0,1916.0,0,2023-08-07 23:59:00,2023-08-08 07:22:00,Spirit Air Lines,Spirit Air Lines,0,3228,late night,14.000000,2.0,76.0,-6.0,3.0,0.0,8.0,17.740741,5.0,82.0,5.0,4.0,0.0,5.0,0.0,West,Mountain,South,South Atlantic,89.7,0.0,0.0,0.0,293635.188,13.5,220.0,25.3,70.0,0.0,0.0,3.0,42322.836,4.9,223.0,18.1
2,3,8,9,3,23,LAS,CLT,0,438.0,1916.0,0,2023-08-09 23:10:00,2023-08-10 06:28:00,Spirit Air Lines,Spirit Air Lines,0,3228,late night,12.375000,1.5,76.0,-6.0,3.0,0.0,8.0,15.185185,5.0,82.0,0.5,4.0,0.0,5.0,0.0,West,Mountain,South,South Atlantic,91.9,0.0,0.0,3.0,234251.969,11.7,239.0,22.6,72.9,0.0,0.0,3.0,52821.523,3.4,212.0,9.4
3,3,8,10,4,23,LAS,CLT,0,438.0,1916.0,0,2023-08-10 23:10:00,2023-08-11 06:28:00,Spirit Air Lines,Spirit Air Lines,0,3228,late night,10.111111,1.0,76.0,-8.5,2.0,0.0,9.0,11.851852,3.0,76.0,-0.5,4.0,0.0,5.0,0.0,West,Mountain,South,South Atlantic,87.6,0.0,0.0,0.0,173884.516,5.9,18.0,9.4,68.7,0.0,0.0,0.0,44947.508,1.1,90.0,1.6
4,3,8,12,6,23,LAS,CLT,0,438.0,1916.0,0,2023-08-12 23:10:00,2023-08-13 06:28:00,Spirit Air Lines,Spirit Air Lines,0,3228,late night,12.250000,3.0,76.0,-5.0,1.0,0.0,8.0,12.961538,4.5,76.0,0.0,4.0,0.0,5.0,0.0,West,Mountain,South,South Atlantic,92.2,0.0,0.0,1.0,265091.875,10.2,50.0,13.0,69.0,0.0,0.0,0.0,40354.332,3.1,176.0,6.5


In [131]:
flights_top20 = missing_stats_filled.drop(columns='route_id')

## Train / Test Split

In [132]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [133]:
train, test = train_test_split(flights_top20, test_size=0.3, random_state=42)

## Encoding Categorical Variables

### Target Encoding for Origin, Destination, Region, Division, and Airline

In [134]:
all_flights = pd.read_csv(DATA_PATH + '/interim/flights_clean.csv')

In [135]:
train_full, test_full = train_test_split(all_flights, test_size=0.3, random_state=42)

In [136]:
train_full['origin'].nunique()

359

#### Calculate target encoding using training set of full flights data

In [137]:
origin_cancellation_rates = train_full.groupby('origin')['cancelled'].mean()
origin_cancellation_rates

origin
ABE    0.005709
ABI    0.010299
ABQ    0.009528
ABR    0.020450
ABY    0.000000
         ...   
XNA    0.010713
XWA    0.026701
YAK    0.046092
YKM    0.000000
YUM    0.007287
Name: cancelled, Length: 359, dtype: float64

In [138]:
destination_cancellation_rates = train_full.groupby('dest')['cancelled'].mean()
destination_cancellation_rates

dest
ABE    0.007479
ABI    0.015625
ABQ    0.008011
ABR    0.019342
ABY    0.001698
         ...   
XNA    0.009178
XWA    0.019426
YAK    0.022044
YKM    0.003460
YUM    0.007389
Name: cancelled, Length: 359, dtype: float64

In [139]:
airline_mkt_cancellation_rate = train_full.groupby('airline_mkt')['cancelled'].mean()
airline_mkt_cancellation_rate

airline_mkt
Alaska Airlines Inc.      0.007541
Allegiant Air             0.006670
American Airlines Inc.    0.011731
Delta Air Lines Inc.      0.012199
Frontier Airlines Inc.    0.021093
Hawaiian Airlines Inc.    0.012863
JetBlue Airways           0.021022
Southwest Airlines Co.    0.010039
Spirit Air Lines          0.016890
United Air Lines Inc.     0.017125
Name: cancelled, dtype: float64

In [140]:
airline_ops_cancellation_rate = train_full.groupby('airline_ops')['cancelled'].mean()
airline_ops_cancellation_rate

airline_ops
Air Wisconsin Airlines Corp                0.013973
Alaska Airlines Inc.                       0.008322
Allegiant Air                              0.006670
American Airlines Inc.                     0.010690
CommuteAir LLC dba CommuteAir              0.014090
Delta Air Lines Inc.                       0.010109
Endeavor Air Inc.                          0.021044
Envoy Air                                  0.009574
Frontier Airlines Inc.                     0.021093
GoJet Airlines LLC d/b/a United Express    0.032348
Hawaiian Airlines Inc.                     0.012863
Horizon Air                                0.007141
JetBlue Airways                            0.021022
Mesa Airlines Inc.                         0.020033
PSA Airlines Inc.                          0.012460
Piedmont Airlines                          0.006782
Republic Airline                           0.027961
SkyWest Airlines Inc.                      0.012144
Southwest Airlines Co.                     0.010039


In [141]:
regions_origion_cancellation_rate = train_full.groupby('region_origin')['cancelled'].mean()
regions_origion_cancellation_rate

region_origin
Midwest           0.012762
Northeast         0.021899
South             0.012193
US Territories    0.012189
West              0.010189
Name: cancelled, dtype: float64

In [142]:
division_origion_cancellation_rate = train_full.groupby('division_origin')['cancelled'].mean()
division_origion_cancellation_rate

division_origin
East North Central    0.012990
East South Central    0.011602
Middle Atlantic       0.022554
Mountain              0.011432
New England           0.019744
Pacific               0.009207
South Atlantic        0.011924
US Territories        0.012189
West North Central    0.012265
West South Central    0.012880
Name: cancelled, dtype: float64

In [143]:
regions_dest_cancellation_rate = train_full.groupby('region_dest')['cancelled'].mean()
regions_dest_cancellation_rate

region_dest
Midwest           0.012853
Northeast         0.021695
South             0.012252
US Territories    0.010030
West              0.010195
Name: cancelled, dtype: float64

In [144]:
division_dest_cancellation_rate = train_full.groupby('division_dest')['cancelled'].mean()
division_dest_cancellation_rate

division_dest
East North Central    0.013199
East South Central    0.011224
Middle Atlantic       0.022576
Mountain              0.011430
New England           0.018802
Pacific               0.009221
South Atlantic        0.011887
US Territories        0.010030
West North Central    0.012095
West South Central    0.013253
Name: cancelled, dtype: float64

### Apply target encoding to modeling dataset

In [145]:
# target encode the training dataset
train['origin_encoded'] = train['origin'].map(origin_cancellation_rates)
train['dest_encoded'] = train['dest'].map(destination_cancellation_rates)
train['airline_mkt_encoded'] = train['airline_mkt'].map(airline_mkt_cancellation_rate)
train['airline_ops_encoded'] = train['airline_ops'].map(airline_ops_cancellation_rate)
train['origin_region_encoded'] = train['region_origin'].map(regions_origion_cancellation_rate)
train['origin_division_encoded'] = train['division_origin'].map(division_origion_cancellation_rate)
train['dest_region_encoded'] = train['region_dest'].map(regions_dest_cancellation_rate)
train['dest_division_encoded'] = train['division_dest'].map(division_dest_cancellation_rate)

In [146]:
train.head()

,quarter,month,day_of_month,day_of_week,hour_of_day,origin,dest,cancelled,scheduled_elapsed_time,distance,code_share_flight,scheduled_departure_datetime,scheduled_arrival_datetime,airline_mkt,airline_ops,is_holiday,departure_window,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_median_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,arr_delay_median_30D,cancelled_sum_30D,div_airport_landings_sum_30D,cancelled_sum_90D,div_airport_landings_sum_90D,region_origin,division_origin,region_dest,division_dest,temperature_2m_origin,precipitation_origin,snow_depth_origin,weather_code_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snow_depth_dest,weather_code_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest,origin_encoded,dest_encoded,airline_mkt_encoded,airline_ops_encoded,origin_region_encoded,origin_division_encoded,dest_region_encoded,dest_division_encoded
1207564,1,1,3,2,14,FLL,LAX,0,177.0,2343.0,0,2023-01-03 14:10:00,2023-01-03 17:07:00,Spirit Air Lines,Spirit Air Lines,0,early afternoon,30.666667,38.0,56.0,21.0,0.0,0.0,3.0,30.666667,38.0,56.0,21.0,0.0,0.0,0.0,0.0,South,South Atlantic,West,Pacific,79.3,0.0,0.0,1.0,59383.203,12.2,122.0,15.2,56.3,0.0,0.0,3.0,45931.758,7.8,87.0,17.0,0.018437,0.008696,0.016890,0.016890,0.012193,0.011924,0.010195,0.009221
913045,2,5,29,1,9,ATL,CLT,0,76.0,226.0,0,2023-05-29 09:40:00,2023-05-29 10:56:00,Delta Air Lines Inc.,Delta Air Lines Inc.,1,morning,-1.941176,-2.0,5.0,-13.0,0.0,0.0,17.0,1.600000,-2.0,80.0,-10.0,0.0,0.0,1.0,0.0,South,South Atlantic,South,South Atlantic,65.0,0.0,0.0,3.0,66601.047,7.9,313.0,11.6,63.7,0.0,0.0,3.0,42979.004,5.7,249.0,6.9,0.008071,0.008567,0.012199,0.010109,0.012193,0.011924,0.012252,0.011887
197561,3,9,16,6,18,LAX,SEA,0,173.0,954.0,0,2023-09-16 18:30:00,2023-09-16 21:23:00,Alaska Airlines Inc.,Alaska Airlines Inc.,0,evening,20.812500,4.0,230.0,-3.0,0.0,0.0,16.0,20.302326,3.0,333.0,-5.0,0.0,0.0,0.0,0.0,West,Pacific,West,Pacific,66.8,0.0,0.0,3.0,51509.188,6.1,276.0,8.5,61.0,0.0,0.0,0.0,74146.984,4.4,300.0,8.1,0.008155,0.007101,0.007541,0.008322,0.010189,0.009207,0.010195,0.009221
710432,4,10,3,2,11,SFO,LAX,0,94.0,337.0,1,2023-10-03 11:32:00,2023-10-03 13:06:00,American Airlines Inc.,SkyWest Airlines Inc.,0,midday,-6.400000,-7.0,5.0,-13.5,0.0,0.0,10.0,-4.233333,-6.0,18.0,-13.0,0.0,0.0,0.0,0.0,West,Pacific,West,Pacific,68.7,0.0,0.0,3.0,78083.992,4.9,60.0,4.9,73.6,0.0,0.0,0.0,71194.227,7.4,263.0,8.1,0.010398,0.008696,0.011731,0.012144,0.010189,0.009207,0.010195,0.009221
1314765,1,2,26,7,18,EWR,DCA,0,83.0,199.0,1,2023-02-26 18:00:00,2023-02-26 19:23:00,United Air Lines Inc.,GoJet Airlines LLC d/b/a United Express,0,evening,39.200000,16.5,150.0,32.5,1.0,0.0,10.0,12.965517,-2.0,150.0,8.0,2.0,0.0,2.0,0.0,Northeast,Middle Atlantic,South,South Atlantic,48.6,0.0,0.0,0.0,155183.734,10.0,226.0,23.7,48.4,0.0,0.0,0.0,104658.797,2.8,256.0,6.3,0.029749,0.015388,0.017125,0.032348,0.021899,0.022554,0.012252,0.011887


In [147]:
# Check correlation between region and division columns
region_division_corr = train[['origin_region_encoded', 'origin_division_encoded', 'dest_region_encoded', 'dest_division_encoded']].corr()
region_division_corr

,origin_region_encoded,origin_division_encoded,dest_region_encoded,dest_division_encoded
origin_region_encoded,1.000000,0.977367,-0.013666,-0.038987
origin_division_encoded,0.977367,1.000000,-0.034409,-0.062415
dest_region_encoded,-0.013666,-0.034409,1.000000,0.966802
dest_division_encoded,-0.038987,-0.062415,0.966802,1.000000


Origin and destination are highly correlated, I'll start by only keeping the more granular data stored in division instead of both features. 

In [148]:
# target encode the testing data
test['origin_encoded'] = test['origin'].map(origin_cancellation_rates)
test['dest_encoded'] = test['dest'].map(destination_cancellation_rates)
test['airline_mkt_encoded'] = test['airline_mkt'].map(airline_mkt_cancellation_rate)
test['airline_ops_encoded'] = test['airline_ops'].map(airline_ops_cancellation_rate)
test['origin_region_encoded'] = test['region_origin'].map(regions_origion_cancellation_rate)
test['origin_division_encoded'] = test['division_origin'].map(division_origion_cancellation_rate)
test['dest_region_encoded'] = test['region_dest'].map(regions_dest_cancellation_rate)
test['dest_division_encoded'] = test['division_dest'].map(division_dest_cancellation_rate)

In [149]:
test.head()

,quarter,month,day_of_month,day_of_week,hour_of_day,origin,dest,cancelled,scheduled_elapsed_time,distance,code_share_flight,scheduled_departure_datetime,scheduled_arrival_datetime,airline_mkt,airline_ops,is_holiday,departure_window,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_median_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,arr_delay_median_30D,cancelled_sum_30D,div_airport_landings_sum_30D,cancelled_sum_90D,div_airport_landings_sum_90D,region_origin,division_origin,region_dest,division_dest,temperature_2m_origin,precipitation_origin,snow_depth_origin,weather_code_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snow_depth_dest,weather_code_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest,origin_encoded,dest_encoded,airline_mkt_encoded,airline_ops_encoded,origin_region_encoded,origin_division_encoded,dest_region_encoded,dest_division_encoded
45129,3,8,6,7,11,LAX,SFO,0,83.0,337.0,0,2023-08-06 11:07:00,2023-08-06 12:30:00,Delta Air Lines Inc.,Delta Air Lines Inc.,0,midday,31.300000,9.0,144.0,8.0,0.0,0.0,10.0,29.928571,0.0,366.0,-5.0,0.0,0.0,0.0,0.0,West,Pacific,West,Pacific,74.2,0.0,0.0,2.0,53149.605,5.4,261.0,8.5,85.8,0.0,0.0,0.0,148950.125,9.9,309.0,12.5,0.008155,0.011123,0.012199,0.010109,0.010189,0.009207,0.010195,0.009221
1362149,1,2,26,7,21,JFK,MCO,0,177.0,944.0,0,2023-02-26 21:55:00,2023-02-27 00:52:00,JetBlue Airways,JetBlue Airways,0,night,43.700000,27.0,154.0,34.5,0.0,0.0,20.0,33.357143,18.5,175.0,8.5,0.0,0.0,0.0,0.0,Northeast,Middle Atlantic,South,South Atlantic,38.8,0.0,0.0,0.0,97112.859,5.9,342.0,13.0,65.0,0.0,0.0,0.0,57742.781,3.1,210.0,6.5,0.023115,0.014283,0.021022,0.021022,0.021899,0.022554,0.012252,0.011887
309312,3,7,15,6,5,DEN,IAH,0,204.0,862.0,0,2023-07-15 05:06:00,2023-07-15 08:30:00,United Air Lines Inc.,United Air Lines Inc.,0,early morning,2.600000,-3.5,36.0,-16.0,0.0,0.0,10.0,-0.461538,-3.0,36.0,-14.0,4.0,0.0,4.0,0.0,West,Mountain,South,West South Central,60.1,0.0,0.0,0.0,63648.293,5.4,204.0,10.3,84.1,0.0,0.0,95.0,46259.844,8.6,208.0,17.4,0.014346,0.012712,0.017125,0.014094,0.010189,0.011432,0.012252,0.013253
1344076,1,2,6,1,9,BOS,ORD,0,118.0,867.0,0,2023-02-06 09:51:00,2023-02-06 11:49:00,American Airlines Inc.,American Airlines Inc.,0,morning,26.347826,-8.0,235.0,-6.0,0.0,0.0,23.0,34.396825,-8.0,630.0,-9.0,0.0,0.0,0.0,0.0,Northeast,New England,Midwest,East North Central,43.0,0.0,0.0,3.0,101049.867,6.8,311.0,16.3,39.7,0.0,0.0,0.0,82349.078,8.1,146.0,13.6,0.018544,0.013344,0.011731,0.010690,0.021899,0.019744,0.012853,0.013199
115619,3,8,2,3,11,DEN,LAS,0,55.0,628.0,0,2023-08-02 11:35:00,2023-08-02 12:30:00,Southwest Airlines Co.,Southwest Airlines Co.,0,midday,23.888889,5.5,186.0,5.5,0.0,0.0,18.0,16.891304,6.5,186.0,4.5,0.0,0.0,0.0,0.0,West,Mountain,West,Mountain,81.3,0.0,0.0,1.0,147637.797,4.7,25.0,6.0,88.1,0.0,0.0,1.0,135170.609,6.5,86.0,6.7,0.014346,0.012155,0.010039,0.010039,0.010189,0.011432,0.010195,0.011430


In [150]:
# drop categorical columns
cols_to_drop = ['origin', 'dest', 'airline_mkt', 'airline_ops', 'region_origin', 'division_origin', 'region_dest', 'division_dest', 'origin_region_encoded', 'dest_region_encoded']

train = train.drop(columns=cols_to_drop)
test = test.drop(columns=cols_to_drop)

In [151]:
# drop datetime columns
train = train.drop(columns=['scheduled_departure_datetime', 'scheduled_arrival_datetime'])
test = test.drop(columns=['scheduled_departure_datetime', 'scheduled_arrival_datetime'])

In [152]:
train.head()

,quarter,month,day_of_month,day_of_week,hour_of_day,cancelled,scheduled_elapsed_time,distance,code_share_flight,is_holiday,departure_window,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_median_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,arr_delay_median_30D,cancelled_sum_30D,div_airport_landings_sum_30D,cancelled_sum_90D,div_airport_landings_sum_90D,temperature_2m_origin,precipitation_origin,snow_depth_origin,weather_code_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snow_depth_dest,weather_code_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest,origin_encoded,dest_encoded,airline_mkt_encoded,airline_ops_encoded,origin_division_encoded,dest_division_encoded
1207564,1,1,3,2,14,0,177.0,2343.0,0,0,early afternoon,30.666667,38.0,56.0,21.0,0.0,0.0,3.0,30.666667,38.0,56.0,21.0,0.0,0.0,0.0,0.0,79.3,0.0,0.0,1.0,59383.203,12.2,122.0,15.2,56.3,0.0,0.0,3.0,45931.758,7.8,87.0,17.0,0.018437,0.008696,0.016890,0.016890,0.011924,0.009221
913045,2,5,29,1,9,0,76.0,226.0,0,1,morning,-1.941176,-2.0,5.0,-13.0,0.0,0.0,17.0,1.600000,-2.0,80.0,-10.0,0.0,0.0,1.0,0.0,65.0,0.0,0.0,3.0,66601.047,7.9,313.0,11.6,63.7,0.0,0.0,3.0,42979.004,5.7,249.0,6.9,0.008071,0.008567,0.012199,0.010109,0.011924,0.011887
197561,3,9,16,6,18,0,173.0,954.0,0,0,evening,20.812500,4.0,230.0,-3.0,0.0,0.0,16.0,20.302326,3.0,333.0,-5.0,0.0,0.0,0.0,0.0,66.8,0.0,0.0,3.0,51509.188,6.1,276.0,8.5,61.0,0.0,0.0,0.0,74146.984,4.4,300.0,8.1,0.008155,0.007101,0.007541,0.008322,0.009207,0.009221
710432,4,10,3,2,11,0,94.0,337.0,1,0,midday,-6.400000,-7.0,5.0,-13.5,0.0,0.0,10.0,-4.233333,-6.0,18.0,-13.0,0.0,0.0,0.0,0.0,68.7,0.0,0.0,3.0,78083.992,4.9,60.0,4.9,73.6,0.0,0.0,0.0,71194.227,7.4,263.0,8.1,0.010398,0.008696,0.011731,0.012144,0.009207,0.009221
1314765,1,2,26,7,18,0,83.0,199.0,1,0,evening,39.200000,16.5,150.0,32.5,1.0,0.0,10.0,12.965517,-2.0,150.0,8.0,2.0,0.0,2.0,0.0,48.6,0.0,0.0,0.0,155183.734,10.0,226.0,23.7,48.4,0.0,0.0,0.0,104658.797,2.8,256.0,6.3,0.029749,0.015388,0.017125,0.032348,0.022554,0.011887


In [153]:
train_encoded = pd.get_dummies(train, prefix='dep_window', drop_first=True, dtype='int')
test_encoded = pd.get_dummies(test, prefix='dep_window', drop_first=True, dtype='int')

In [154]:
train_encoded.head()

,quarter,month,day_of_month,day_of_week,hour_of_day,cancelled,scheduled_elapsed_time,distance,code_share_flight,is_holiday,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_median_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,arr_delay_median_30D,cancelled_sum_30D,div_airport_landings_sum_30D,cancelled_sum_90D,div_airport_landings_sum_90D,temperature_2m_origin,precipitation_origin,snow_depth_origin,weather_code_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snow_depth_dest,weather_code_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest,origin_encoded,dest_encoded,airline_mkt_encoded,airline_ops_encoded,origin_division_encoded,dest_division_encoded,dep_window_early afternoon,dep_window_early morning,dep_window_evening,dep_window_late night,dep_window_midday,dep_window_morning,dep_window_night,dep_window_overnight
1207564,1,1,3,2,14,0,177.0,2343.0,0,0,30.666667,38.0,56.0,21.0,0.0,0.0,3.0,30.666667,38.0,56.0,21.0,0.0,0.0,0.0,0.0,79.3,0.0,0.0,1.0,59383.203,12.2,122.0,15.2,56.3,0.0,0.0,3.0,45931.758,7.8,87.0,17.0,0.018437,0.008696,0.016890,0.016890,0.011924,0.009221,1,0,0,0,0,0,0,0
913045,2,5,29,1,9,0,76.0,226.0,0,1,-1.941176,-2.0,5.0,-13.0,0.0,0.0,17.0,1.600000,-2.0,80.0,-10.0,0.0,0.0,1.0,0.0,65.0,0.0,0.0,3.0,66601.047,7.9,313.0,11.6,63.7,0.0,0.0,3.0,42979.004,5.7,249.0,6.9,0.008071,0.008567,0.012199,0.010109,0.011924,0.011887,0,0,0,0,0,1,0,0
197561,3,9,16,6,18,0,173.0,954.0,0,0,20.812500,4.0,230.0,-3.0,0.0,0.0,16.0,20.302326,3.0,333.0,-5.0,0.0,0.0,0.0,0.0,66.8,0.0,0.0,3.0,51509.188,6.1,276.0,8.5,61.0,0.0,0.0,0.0,74146.984,4.4,300.0,8.1,0.008155,0.007101,0.007541,0.008322,0.009207,0.009221,0,0,1,0,0,0,0,0
710432,4,10,3,2,11,0,94.0,337.0,1,0,-6.400000,-7.0,5.0,-13.5,0.0,0.0,10.0,-4.233333,-6.0,18.0,-13.0,0.0,0.0,0.0,0.0,68.7,0.0,0.0,3.0,78083.992,4.9,60.0,4.9,73.6,0.0,0.0,0.0,71194.227,7.4,263.0,8.1,0.010398,0.008696,0.011731,0.012144,0.009207,0.009221,0,0,0,0,1,0,0,0
1314765,1,2,26,7,18,0,83.0,199.0,1,0,39.200000,16.5,150.0,32.5,1.0,0.0,10.0,12.965517,-2.0,150.0,8.0,2.0,0.0,2.0,0.0,48.6,0.0,0.0,0.0,155183.734,10.0,226.0,23.7,48.4,0.0,0.0,0.0,104658.797,2.8,256.0,6.3,0.029749,0.015388,0.017125,0.032348,0.022554,0.011887,0,0,1,0,0,0,0,0


## Scaling Numerical Columns

In [155]:
X_train = train_encoded.drop(columns='cancelled')
y_train = train_encoded['cancelled']

X_test = test_encoded.drop(columns='cancelled')
y_test = test_encoded['cancelled']

In [156]:
X_train.head()

,quarter,month,day_of_month,day_of_week,hour_of_day,scheduled_elapsed_time,distance,code_share_flight,is_holiday,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_median_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,arr_delay_median_30D,cancelled_sum_30D,div_airport_landings_sum_30D,cancelled_sum_90D,div_airport_landings_sum_90D,temperature_2m_origin,precipitation_origin,snow_depth_origin,weather_code_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snow_depth_dest,weather_code_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest,origin_encoded,dest_encoded,airline_mkt_encoded,airline_ops_encoded,origin_division_encoded,dest_division_encoded,dep_window_early afternoon,dep_window_early morning,dep_window_evening,dep_window_late night,dep_window_midday,dep_window_morning,dep_window_night,dep_window_overnight
1207564,1,1,3,2,14,177.0,2343.0,0,0,30.666667,38.0,56.0,21.0,0.0,0.0,3.0,30.666667,38.0,56.0,21.0,0.0,0.0,0.0,0.0,79.3,0.0,0.0,1.0,59383.203,12.2,122.0,15.2,56.3,0.0,0.0,3.0,45931.758,7.8,87.0,17.0,0.018437,0.008696,0.016890,0.016890,0.011924,0.009221,1,0,0,0,0,0,0,0
913045,2,5,29,1,9,76.0,226.0,0,1,-1.941176,-2.0,5.0,-13.0,0.0,0.0,17.0,1.600000,-2.0,80.0,-10.0,0.0,0.0,1.0,0.0,65.0,0.0,0.0,3.0,66601.047,7.9,313.0,11.6,63.7,0.0,0.0,3.0,42979.004,5.7,249.0,6.9,0.008071,0.008567,0.012199,0.010109,0.011924,0.011887,0,0,0,0,0,1,0,0
197561,3,9,16,6,18,173.0,954.0,0,0,20.812500,4.0,230.0,-3.0,0.0,0.0,16.0,20.302326,3.0,333.0,-5.0,0.0,0.0,0.0,0.0,66.8,0.0,0.0,3.0,51509.188,6.1,276.0,8.5,61.0,0.0,0.0,0.0,74146.984,4.4,300.0,8.1,0.008155,0.007101,0.007541,0.008322,0.009207,0.009221,0,0,1,0,0,0,0,0
710432,4,10,3,2,11,94.0,337.0,1,0,-6.400000,-7.0,5.0,-13.5,0.0,0.0,10.0,-4.233333,-6.0,18.0,-13.0,0.0,0.0,0.0,0.0,68.7,0.0,0.0,3.0,78083.992,4.9,60.0,4.9,73.6,0.0,0.0,0.0,71194.227,7.4,263.0,8.1,0.010398,0.008696,0.011731,0.012144,0.009207,0.009221,0,0,0,0,1,0,0,0
1314765,1,2,26,7,18,83.0,199.0,1,0,39.200000,16.5,150.0,32.5,1.0,0.0,10.0,12.965517,-2.0,150.0,8.0,2.0,0.0,2.0,0.0,48.6,0.0,0.0,0.0,155183.734,10.0,226.0,23.7,48.4,0.0,0.0,0.0,104658.797,2.8,256.0,6.3,0.029749,0.015388,0.017125,0.032348,0.022554,0.011887,0,0,1,0,0,0,0,0


In [157]:
print('Train shape: ', X_train.shape, y_train.shape)
print('Test shape: ', X_test.shape, y_test.shape)

Train shape:  (1044712, 54) (1044712,)
Test shape:  (447735, 54) (447735,)


In [159]:
cat_cols = ['code_share_flight', 'is_holiday', 'dep_window_early afternoon', 'dep_window_early morning', 'dep_window_evening', 'dep_window_late night', 'dep_window_midday', 'dep_window_morning',	'dep_window_night',	'dep_window_overnight']

X_train_cat = X_train[cat_cols]
X_test_cat = X_test[cat_cols]

X_train_num = X_train.drop(columns=cat_cols)
X_test_num = X_test.drop(columns=cat_cols)

In [160]:
X_train_cat.head()

,code_share_flight,is_holiday,dep_window_early afternoon,dep_window_early morning,dep_window_evening,dep_window_late night,dep_window_midday,dep_window_morning,dep_window_night,dep_window_overnight
1207564,0,0,1,0,0,0,0,0,0,0
913045,0,1,0,0,0,0,0,1,0,0
197561,0,0,0,0,1,0,0,0,0,0
710432,1,0,0,0,0,0,1,0,0,0
1314765,1,0,0,0,1,0,0,0,0,0


In [161]:
X_train_num.head()

,quarter,month,day_of_month,day_of_week,hour_of_day,scheduled_elapsed_time,distance,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_median_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,arr_delay_median_30D,cancelled_sum_30D,div_airport_landings_sum_30D,cancelled_sum_90D,div_airport_landings_sum_90D,temperature_2m_origin,precipitation_origin,snow_depth_origin,weather_code_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snow_depth_dest,weather_code_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest,origin_encoded,dest_encoded,airline_mkt_encoded,airline_ops_encoded,origin_division_encoded,dest_division_encoded
1207564,1,1,3,2,14,177.0,2343.0,30.666667,38.0,56.0,21.0,0.0,0.0,3.0,30.666667,38.0,56.0,21.0,0.0,0.0,0.0,0.0,79.3,0.0,0.0,1.0,59383.203,12.2,122.0,15.2,56.3,0.0,0.0,3.0,45931.758,7.8,87.0,17.0,0.018437,0.008696,0.016890,0.016890,0.011924,0.009221
913045,2,5,29,1,9,76.0,226.0,-1.941176,-2.0,5.0,-13.0,0.0,0.0,17.0,1.600000,-2.0,80.0,-10.0,0.0,0.0,1.0,0.0,65.0,0.0,0.0,3.0,66601.047,7.9,313.0,11.6,63.7,0.0,0.0,3.0,42979.004,5.7,249.0,6.9,0.008071,0.008567,0.012199,0.010109,0.011924,0.011887
197561,3,9,16,6,18,173.0,954.0,20.812500,4.0,230.0,-3.0,0.0,0.0,16.0,20.302326,3.0,333.0,-5.0,0.0,0.0,0.0,0.0,66.8,0.0,0.0,3.0,51509.188,6.1,276.0,8.5,61.0,0.0,0.0,0.0,74146.984,4.4,300.0,8.1,0.008155,0.007101,0.007541,0.008322,0.009207,0.009221
710432,4,10,3,2,11,94.0,337.0,-6.400000,-7.0,5.0,-13.5,0.0,0.0,10.0,-4.233333,-6.0,18.0,-13.0,0.0,0.0,0.0,0.0,68.7,0.0,0.0,3.0,78083.992,4.9,60.0,4.9,73.6,0.0,0.0,0.0,71194.227,7.4,263.0,8.1,0.010398,0.008696,0.011731,0.012144,0.009207,0.009221
1314765,1,2,26,7,18,83.0,199.0,39.200000,16.5,150.0,32.5,1.0,0.0,10.0,12.965517,-2.0,150.0,8.0,2.0,0.0,2.0,0.0,48.6,0.0,0.0,0.0,155183.734,10.0,226.0,23.7,48.4,0.0,0.0,0.0,104658.797,2.8,256.0,6.3,0.029749,0.015388,0.017125,0.032348,0.022554,0.011887


In [162]:
scaler = StandardScaler()

scaler.fit(X_train_num)

X_train_num_scaled = pd.DataFrame(scaler.transform(X_train_num), columns=X_train_num.columns, index=X_train_num.index)
X_test_num_scaled = pd.DataFrame(scaler.transform(X_test_num), columns=X_test_num.columns, index=X_test_num.index)

In [163]:
X_train_merged = pd.concat([X_train_num_scaled, X_train_cat], axis=1)
X_test_merged = pd.concat([X_test_num_scaled, X_test_cat], axis=1)

In [164]:
X_train_merged

,quarter,month,day_of_month,day_of_week,hour_of_day,scheduled_elapsed_time,distance,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_median_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,arr_delay_median_30D,cancelled_sum_30D,div_airport_landings_sum_30D,cancelled_sum_90D,div_airport_landings_sum_90D,temperature_2m_origin,precipitation_origin,snow_depth_origin,weather_code_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snow_depth_dest,weather_code_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest,origin_encoded,dest_encoded,airline_mkt_encoded,airline_ops_encoded,origin_division_encoded,dest_division_encoded,code_share_flight,is_holiday,dep_window_early afternoon,dep_window_early morning,dep_window_evening,dep_window_late night,dep_window_midday,dep_window_morning,dep_window_night,dep_window_overnight
1207564,-1.361845,-1.620543,-1.452345,-0.984862,0.189847,-0.031459,1.937362,0.617993,2.272436,-0.369399,1.263474,-0.320950,-0.02657,-1.252609,0.753796,2.917779,-0.651917,1.595429,-0.455432,-0.045443,-0.614674,-0.075479,0.942484,-0.059526,-0.098763,-0.279899,-0.691314,1.486775,-0.662031,0.439144,-0.384728,-0.061437,-0.097708,-0.151443,-0.923607,0.334950,-1.025207,0.658845,0.725836,-0.927173,0.758972,0.686502,-0.373972,-1.033742,0,0,1,0,0,0,0,0,0,0
913045,-0.462343,-0.450140,1.515007,-1.485173,-0.797123,-0.935554,-1.309607,-0.714057,-0.334788,-0.688254,-0.571252,-0.320950,-0.02657,0.282212,-0.720075,-0.326840,-0.544094,-0.447513,-0.455432,-0.045443,-0.247148,-0.075479,0.140555,-0.059526,-0.098763,-0.156940,-0.587775,0.382257,1.248121,-0.070664,0.032863,-0.061437,-0.097708,-0.151443,-0.965718,-0.207061,0.605140,-0.767828,-0.917549,-0.948258,-0.513727,-0.710339,-0.373972,-0.389142,0,1,0,0,0,0,0,1,0,0
197561,0.437160,0.720262,0.031331,1.016382,0.979423,-0.067265,-0.193030,0.215444,0.056296,0.718457,-0.031627,-0.320950,-0.02657,0.172582,0.228256,0.078737,0.592533,-0.118006,-0.455432,-0.045443,-0.614674,-0.075479,0.241497,-0.059526,-0.098763,-0.156940,-0.804266,-0.080100,0.878091,-0.509665,-0.119502,-0.061437,-0.097708,-0.336215,-0.521212,-0.542592,1.118397,-0.598323,-0.904235,-1.189224,-1.777531,-1.078380,-1.020498,-1.033742,0,0,0,0,1,0,0,0,0,0
710432,1.336662,1.012863,-1.452345,-0.984862,-0.402335,-0.774428,-1.139360,-0.896203,-0.660691,-0.688254,-0.598233,-0.320950,-0.02657,-0.485198,-1.015863,-0.651302,-0.822635,-0.645217,-0.455432,-0.045443,-0.614674,-0.075479,0.348047,-0.059526,-0.098763,-0.156940,-0.423054,-0.388337,-1.282081,-1.019473,0.591532,-0.061437,-0.097708,-0.336215,-0.563323,0.231709,0.746034,-0.598323,-0.548545,-0.927173,-0.640548,-0.291071,-1.020498,-1.033742,1,0,0,0,0,0,1,0,0,0
1314765,-1.361845,-1.327942,1.172621,1.516693,0.979423,-0.872894,-1.351018,0.966584,0.871053,0.218293,1.884043,1.237885,-0.02657,-0.485198,-0.143768,-0.326840,-0.229612,0.738711,1.070058,-0.045443,0.120378,-0.075479,-0.779140,-0.059526,-0.098763,-0.341378,0.682932,0.921673,0.378051,1.642857,-0.830535,-0.061437,-0.097708,-0.336215,-0.086064,-0.955553,0.675587,-0.852581,2.519309,0.172827,0.822719,3.870703,2.156194,-0.389142,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,0.437160,0.135061,-0.767571,1.516693,0.584635,-0.344759,-0.349473,0.059701,0.382199,-0.225602,0.076298,-0.320950,-0.02657,-0.485198,0.763938,0.565430,-0.126283,0.573958,-0.455432,-0.045443,0.120378,-0.075479,1.441586,-0.059526,-0.098763,-0.341378,-0.187738,0.305197,-1.602106,-0.481342,1.133272,-0.061437,-0.097708,-0.336215,0.770194,1.909362,0.444118,1.478122,0.267476,-0.262271,-0.640548,-0.590578,-0.146219,-0.338870,0,0,0,0,0,0,0,0,0,0
1414414,-1.361845,-1.035342,1.629136

In [165]:
X_test_merged

,quarter,month,day_of_month,day_of_week,hour_of_day,scheduled_elapsed_time,distance,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_median_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,arr_delay_median_30D,cancelled_sum_30D,div_airport_landings_sum_30D,cancelled_sum_90D,div_airport_landings_sum_90D,temperature_2m_origin,precipitation_origin,snow_depth_origin,weather_code_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snow_depth_dest,weather_code_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest,origin_encoded,dest_encoded,airline_mkt_encoded,airline_ops_encoded,origin_division_encoded,dest_division_encoded,code_share_flight,is_holiday,dep_window_early afternoon,dep_window_early morning,dep_window_evening,dep_window_late night,dep_window_midday,dep_window_morning,dep_window_night,dep_window_overnight
45129,0.437160,0.427661,-1.109958,1.516693,-0.402335,-0.872894,-1.139360,0.643865,0.382199,0.180781,0.561961,-0.320950,-0.02657,-0.485198,0.716370,-0.164610,0.740789,-0.118006,-0.455432,-0.045443,-0.614674,-0.075479,0.656481,-0.059526,-0.098763,-0.218419,-0.780734,-0.259905,0.728079,-0.509665,1.279993,-0.061437,-0.097708,-0.336215,0.545602,0.876960,1.208972,0.023198,-0.904235,-0.528224,-0.513727,-0.710339,-1.020498,-1.033742,0,0,0,0,0,0,1,0,0,0
1362149,-1.361845,-1.327942,1.172621,1.516693,1.571605,-0.031459,-0.208368,1.150412,1.555449,0.243302,1.991968,-0.320950,-0.02657,0.611102,0.890221,1.336027,-0.117298,0.771662,-0.455432,-0.045443,-0.614674,-0.075479,-1.328714,-0.059526,-0.098763,-0.341378,-0.150087,-0.131473,1.538144,0.127595,0.106223,-0.061437,-0.097708,-0.336215,-0.755163,-0.878123,0.212649,-0.824330,1.467561,-0.008789,1.880168,1.537749,2.156194,-0.389142,0,0,0,0,0,0,0,0,1,0
309312,0.437160,0.135061,-0.082798,1.016382,-1.586700,0.210229,-0.334136,-0.528547,-0.432559,-0.494440,-0.733139,-0.320950,-0.02657,-0.485198,-0.824608,-0.407956,-0.741768,-0.711119,2.595548,-0.045443,0.855429,-0.075479,-0.134232,-0.059526,-0.098763,-0.341378,-0.630132,-0.259905,0.158034,-0.254761,1.184060,-0.061437,-0.097708,5.514924,-0.918928,0.541430,0.192521,0.715346,0.077256,-0.267134,0.822719,0.110557,-0.490858,-0.058768,0,0,0,1,0,0,0,0,0,0
1344076,-1.361845,-1.327942,-1.109958,-1.485173,-0.797123,-0.559594,-0.326467,0.441565,-0.725871,0.749718,-0.193514,-0.320950,-0.02657,0.939992,0.942940,-0.813533,1.926834,-0.381612,-0.455432,-0.045443,-0.614674,-0.075479,-1.093182,-0.059526,-0.098763,-0.156940,-0.093611,0.099706,1.228119,0.594919,-1.321487,-0.061437,-0.097708,-0.336215,-0.404237,0.412380,-0.431438,0.178578,0.742800,-0.163210,-0.640548,-0.590578,1.487345,-0.071679,0,0,0,0,0,0,0,1,0,0
115619,0.437160,0.427661,-1.566474,-0.484551,-0.402335,-1.123534,-0.693036,0.341116,0.154067,0.443367,0.427055,-0.320950,-0.02657,0.391842,0.055295,0.362641,-0.067879,0.508057,-0.455432,-0.045443,-0.614674,-0.075479,1.054642,-0.059526,-0.098763,-0.279899,0.574686,-0.439710,-1.632109,-0.863698,1.409785,-0.061437,-0.097708,-0.274624,0.349084,-0.000581,-1.035270,-0.796079,0.077256,-0.358570,-1.099822,-0.724754,-0.490858,-0.499668,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310777,0.437160,0.135061,-0.311055,0.015760,-1.389305,-0.837088,0.458818,-0.660631,-0.367378,-0.531953,-0.598233,-0.320950,-0.02657,1.707402,-0.425179,-0.245725,0.394859,-0.249809,0.307313,-0.045443,0.487904,-0.075479,1.093897,-0.059526,-0.098763,-0.279899,-0.874861,-0.979126,0.118031,0.538273,0.049792,-0.061437,-0.097708,-0.336215,-0.998471,-1.497564,1.541080,-1.459976,-0.409022,-0.927173,0.822719,0.110557,-0.146219,-1.033742,0,0,0,0,0,0,0,1,0,0
1343908,-1.361845,-1.327942,1.400878,-0.98486

In [166]:
y_train

1207564    0
913045     0
197561     0
710432     0
1314765    0
          ..
259178     0
1414414    0
131932     0
671155     0
121958     0
Name: cancelled, Length: 1044712, dtype: int64

In [167]:
X_train_merged.to_csv(DATA_PATH + '/modeling/X_train.csv', index=False)
X_test_merged.to_csv(DATA_PATH + '/modeling/X_test.csv', index=False)
y_train.to_csv(DATA_PATH + '/modeling/y_train.csv', index=False)
y_test.to_csv(DATA_PATH + '/modeling/y_test.csv', index=False)